<a href="https://colab.research.google.com/github/navidadkhah/RAG/blob/main/Creating_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 12.7 MB/s eta

In [4]:
import chromadb
import pandas as pd
import numpy as np

In [5]:
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/RAG/Vectordb")
collection = chroma_client.get_or_create_collection(name="my_collection")

In [6]:
path = "/content/drive/MyDrive/RAG/movies_&_series.csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Name,overview,genres,language,popularity
0,0,Havoc,When a drug heist swerves lethally out of cont...,"Action, Crime, Thriller",en,575.1039
1,1,A Working Man,Levon Cade left behind a decorated military ca...,"Action, Crime, Thriller",en,548.9128
2,2,新幹線大爆破,When panic erupts on a Tokyo-bound bullet trai...,"Action, Thriller, Crime, Drama",ja,276.1329
3,3,In the Lost Lands,A queen sends the powerful and feared sorceres...,"Action, Fantasy, Adventure",en,275.3451
4,4,ज्वेल थीफ: द हीस्ट बिगिन्स,"In this high-octane battle of wits and wills, ...","Action, Thriller",hi,273.4631


In [7]:
df['combined_text'] = (
    "Title: " + df['Name'].astype(str) + ". " +
    "Overview: " + df['overview'].astype(str) + ". " +
    "Genres: " + df['genres'].astype(str) + ". " +
    "Language: " + df['language'].astype(str) + ". " +
    "Popularity: " + df['popularity'].astype(str)
)
df.head()

,Unnamed: 0,Name,overview,genres,language,popularity,combined_text
0,0,Havoc,When a drug heist swerves lethally out of cont...,"Action, Crime, Thriller",en,575.1039,Title: Havoc. Overview: When a drug heist swer...
1,1,A Working Man,Levon Cade left behind a decorated military ca...,"Action, Crime, Thriller",en,548.9128,Title: A Working Man. Overview: Levon Cade lef...
2,2,新幹線大爆破,When panic erupts on a Tokyo-bound bullet trai...,"Action, Thriller, Crime, Drama",ja,276.1329,Title: 新幹線大爆破. Overview: When panic erupts on ...
3,3,In the Lost Lands,A queen sends the powerful and feared sorceres...,"Action, Fantasy, Adventure",en,275.3451,Title: In the Lost Lands. Overview: A queen se...
4,4,ज्वेल थीफ: द हीस्ट बिगिन्स,"In this high-octane battle of wits and wills, ...","Action, Thriller",hi,273.4631,Title: ज्वेल थीफ: द हीस्ट बिगिन्स. Overview: I...


In [11]:
df["combined_text"][:5].to_list()

["Title: Havoc. Overview: When a drug heist swerves lethally out of control, a jaded cop fights his way through a corrupt city's criminal underworld to save a politician's son.. Genres: Action, Crime, Thriller. Language: en. Popularity: 575.1039",
 "Title: A Working Man. Overview: Levon Cade left behind a decorated military career in the black ops to live a simple life working construction. But when his boss's daughter, who is like family to him, is taken by human traffickers, his search to bring her home uncovers a world of corruption far greater than he ever could have imagined.. Genres: Action, Crime, Thriller. Language: en. Popularity: 548.9128",
 'Title: 新幹線大爆破. Overview: When panic erupts on a Tokyo-bound bullet train that will explode if it slows below 100 kph, authorities race against time to save everyone on board.. Genres: Action, Thriller, Crime, Drama. Language: ja. Popularity: 276.1329',
 'Title: In the Lost Lands. Overview: A queen sends the powerful and feared sorceress 

In [12]:
vectors = df["combined_text"][:5].to_list()
collection.add(
    documents=vectors,
    ids=["id1","id2","id3","id4","id5"]
    )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 103MiB/s]


In [13]:
print(collection.count())

5


In [14]:
collection.peek()

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5'],
 'embeddings': array([[ 0.03738717,  0.04748575, -0.05010892, ...,  0.03359606,
          0.02930197, -0.01267304],
        [-0.13455388, -0.02684624, -0.10108222, ..., -0.03512937,
         -0.0144708 ,  0.00107189],
        [-0.00158789, -0.01445741,  0.03929684, ...,  0.00343756,
         -0.0712861 ,  0.03635733],
        [-0.00597597,  0.01764174, -0.01465255, ..., -0.02730393,
         -0.05556757, -0.02783172],
        [-0.03257563,  0.07209565, -0.06889339, ..., -0.0201755 ,
         -0.06639002, -0.0510019 ]]),
 'documents': ["Title: Havoc. Overview: When a drug heist swerves lethally out of control, a jaded cop fights his way through a corrupt city's criminal underworld to save a politician's son.. Genres: Action, Crime, Thriller. Language: en. Popularity: 575.1039",
  "Title: A Working Man. Overview: Levon Cade left behind a decorated military career in the black ops to live a simple life working construction. But when his boss's 

In [16]:
results = collection.query(
    query_texts=["I want a movie in a lost world"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
results

{'ids': [['id4', 'id5']],
 'embeddings': None,
 'documents': [['Title: In the Lost Lands. Overview: A queen sends the powerful and feared sorceress Gray Alys to the ghostly wilderness of the Lost Lands in search of a magical power, where she and her guide, the drifter Boyce, must outwit and outfight both man and demon.. Genres: Action, Fantasy, Adventure. Language: en. Popularity: 275.3451',
   'Title: ज्वेल थीफ: द हीस्ट बिगिन्स. Overview: In this high-octane battle of wits and wills, ingenious con artist Rehan devises a diamond heist while trying to outsmart Rajan, his sadistic adversary.. Genres: Action, Thriller. Language: hi. Popularity: 273.4631']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.2160332202911377, 1.5710945129394531]]}